# Q&A Generation with OpenAI GPT

This Jupyter notebook demonstrates how to:
- Load an OpenAI API key from a `.env` file
- Generate three clear Spanish Q&A pairs from BHL-extracted text
- Focus Q&A on a specific species (e.g., Dermanura azteca)
- Track token usage and estimate costs

In [22]:
!pip install openai
!pip install openai python-dotenv

 ## Imports and Configuration
Load environment variables from `.env` and initialize the OpenAI client.

In [9]:
import os
from dotenv import load_dotenv
import openai

In [24]:
# Load environment variables from .env
load_dotenv()

# Set API key
openai.api_key = os.getenv("OPENAI_API_KEY")
assert openai.api_key, "Please set OPENAI_API_KEY in your .env file."


## Core Helpers
These functions build the prompt from BHL text and optional species context.

In [25]:

def _call_chat(model: str, prompt: str, max_tokens: int, temperature: float):
    messages = [
        {"role": "system", "content": "Eres un asistente útil para generar preguntas y respuestas en español."},
        {"role": "user", "content": prompt}
    ]
    return openai.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

## 1. BHL Text Q&A
Generate exactly three Q&A pairs from a BHL-extracted text.

In [26]:
def generate_bhl_qna(text: str,
                     model: str = "gpt-3.5-turbo",
                     max_tokens: int = 500,
                     temperature: float = 0.3) -> str:
    """
    Returns three Q&A pairs (preguntas y respuestas) in Spanish based on BHL `text`.
    """
    prompt = (
        f"Hola, tengo este texto extraído desde la API de BHL y quiero generar tres preguntas con sus respectivas respuestas.\n"
        f"Este es el texto:\n\n{text}\n\n"
        "Por favor, entrega exactamente tres preguntas en español con sus respectivas respuestas, en un formato claro."
    )
    response = _call_chat(model, prompt, max_tokens, temperature)
    return response.choices[0].message.content.strip()

## 2. BHL Text Q&A with Cost
Returns Q&A plus usage and estimated cost.

In [27]:
def generate_bhl_qna_with_cost(text: str,
                               model: str = "gpt-3.5-turbo",
                               max_tokens: int = 500,
                               temperature: float = 0.3) -> tuple:
    """
    Returns (qna_text, usage, cost_estimate).
    """
    prompt = (
        f"Hola, tengo este texto extraído desde la API de BHL y quiero generar tres preguntas con sus respectivas respuestas.\n"
        f"Este es el texto:\n\n{text}\n\n"
        "Por favor, entrega exactamente tres preguntas en español con sus respectivas respuestas, en un formato claro."
    )
    response = _call_chat(model, prompt, max_tokens, temperature)
    usage = response.usage
    cost = (usage.total_tokens * 0.002) / 1000
    return response.choices[0].message.content.strip(), usage, cost

## 3. Species-Oriented Q&A
Generate Q&A focused on a specific species.

In [28]:
def generate_species_qna(text: str,
                         species: str,
                         model: str = "gpt-4",
                         max_tokens: int = 500,
                         temperature: float = 0.3) -> str:
    """
    Returns three Q&A pairs oriented around `species` from BHL `text`.
    """
    prompt = (
        f"Usa el siguiente texto y genera exactamente tres preguntas en español con sus respuestas, "
        f"orientadas a información sobre la especie: {species}.\n\nTexto:\n{text}"
    )
    response = _call_chat(model, prompt, max_tokens, temperature)
    return response.choices[0].message.content.strip()

## 4. Species-Oriented Q&A with Cost
Returns species-focused Q&A plus usage and estimated cost.

In [29]:
def generate_species_qna_with_cost(text: str,
                                   species: str,
                                   model: str = "gpt-4",
                                   max_tokens: int = 500,
                                   temperature: float = 0.3) -> tuple:
    """
    Returns (qna_text, usage, cost_estimate) for species-focused Q&A.
    """
    prompt = (
        f"Usa el siguiente texto y genera exactamente tres preguntas en español con sus respuestas, "
        f"orientadas a información sobre la especie: {species}.\n\nTexto:\n{text}"
    )
    response = _call_chat(model, prompt, max_tokens, temperature)
    usage = response.usage
    cost = (usage.total_tokens * 0.002) / 1000
    return response.choices[0].message.content.strip(), usage, cost

# Usage

In [31]:
text_content = """
Mesa Montero Es una zona que se presenta en la parte oeste del Estado, entre las Sierras Fría, El Piñal, Sierra del Laurel y El Valle de Aguascal lentes. La vegetación es un pastizal con algunas áreas de encinares. Los mamíferos de la localidad son: coyote, venado cola blanca, ardillón, ratones de patas blancas, zorrillos, liebre de cola negra y zorra gris (Proa, 1 982; De la Riva et al., 2000b). Serranía Juan El Grande Es una serranía localizada en El Llano, frente al poblado de Palo Alto. La vegetación presente es encinares en la parte alta y matorral crasicaule en la zona baja y los mamíferos de la zona son: diversas especies de ratas canguro, varias especies de ratones de patas blancas, conejo cola blanca y liebre cola negra (Proa, 1 982). Serranía de Tepezalá Se localiza en la porción noreste del Estado. Se caracteriza por presentar una vegetación de matorral crasicaule, pastizal y encinares en las partes altas. Los mamíferos presentes en la zona son: coyote, zorrillos, conejo de cola blanca, liebre cola negra, diversas especies de ratones de abazones, ratones de patas blancas, rata magueyera, ardillas (figura 3.18.3), ardillones. De la Riva (1989) realizó un estudio de mamíferos con algunas colectas en la cercanía de esta sierra, sin embargo, creemos que es necesario estudiarla mejor. Sierra de Guajolotes Se localiza en la parte noroeste de la ciudad de Aguascalientes; es una sierra con topografía accidentada y una vegetación representada por bosque de encino en sus partes altas y chaparral de encino y manzanita en las partes bajas. Se tiene poca o nula información sobre las especies de mamíferos que se presentan en esta sierra. Es necesario que los próximos estudios sobre mamíferos estén enfocados a esta zona con el fin de conocer su mastofauna representativa. Valle de Huejúcar Es el valle por donde se desplaza la carretera 70 y es donde está asentada la ciudad de Calvillo. Se caracteriza por ser una zona donde predominan los cultivos de guayaba y en las áreas naturales la vegetación es matorral subtropical. La zona está habitada por diferentes especies de mamíferos como: coyote, ardillón, liebre cola negra, conejo cola blanca, tlacuache, diversas especies de ratones de bolsas, murciélago vampiro.
"""

# # BHL Q&A
#qna = generate_bhl_qna(text_content)

# # BHL Q&A with cost
#qna, usage, cost = generate_bhl_qna_with_cost(text_content)

# # Species Q&A
#qna_spec = generate_species_qna(text_content, species="Dermanura azteca")

# # Species Q&A with cost
qna_spec, usage_spec, cost_spec = generate_species_qna_with_cost(text_content, species="Dermanura azteca")

## Displaying Species Q&A with Readable Output
This block formats and prints the species-oriented Q&A, usage breakdown, and estimated cost neatly.

In [35]:
# Ensure you've already run the species Q&A with cost cell
print("=== Species-Oriented Q&A ===")
print(qna_spec)

print("=== Token Usage Breakdown ===")
print(f"Prompt tokens:     {usage_spec.prompt_tokens}")
print(f"Completion tokens: {usage_spec.completion_tokens}")
print(f"Total tokens:      {usage_spec.total_tokens}")

print(f"Estimated Cost:    ${cost_spec:.6f}")

=== Species-Oriented Q&A ===
Pregunta 1: ¿Qué tipo de vegetación se encuentra en la zona de Mesa Montero?
Respuesta 1: En la zona de Mesa Montero, la vegetación es principalmente un pastizal con algunas áreas de encinares.

Pregunta 2: ¿Qué especies de mamíferos se pueden encontrar en la Serranía de Tepezalá?
Respuesta 2: En la Serranía de Tepezalá se pueden encontrar mamíferos como el coyote, zorrillos, conejo de cola blanca, liebre cola negra, diversas especies de ratones de abazones, ratones de patas blancas, rata magueyera, ardillas y ardillones.

Pregunta 3: ¿Qué caracteriza al Valle de Huejúcar en términos de vegetación y fauna?
Respuesta 3: El Valle de Huejúcar se caracteriza por ser una zona donde predominan los cultivos de guayaba y en las áreas naturales la vegetación es matorral subtropical. La zona está habitada por diferentes especies de mamíferos como: coyote, ardillón, liebre cola negra, conejo cola blanca, tlacuache, diversas especies de ratones de bolsas y murciélago v